# 05 - Context Injection: Pass Data to the Sandbox

The sandbox is secure, but how do we give it data to work with?

**Context injection** lets you pass variables directly into sandbox code, enabling:

- API responses to be processed
- Database results to be transformed
- User data to be filtered
- Configuration to drive behavior

## Learning Objectives

After this notebook, you will:

- [ ] Inject variables into sandbox execution
- [ ] Process external data safely
- [ ] Validate context before use

---

## How Context Injection Works

When you call `sandbox.execute(code, context)`, each key in the context object becomes a
**variable** in the sandbox:

```typescript
sandbox.execute(code, {
  name: "Alice",
  items: [1, 2, 3],
});

// Inside sandbox, these are available:
// const name = "Alice";
// const items = [1, 2, 3];
```

This is secure because:

- Values are **serialized** (JSON only, no functions)
- Variable names are **validated** (alphanumeric + underscore)
- Injection is **one-way** (sandbox can't modify the original)

In [ ]:
// Setup sandbox
const { DenoSandboxExecutor } = await import("../../src/sandbox/executor.ts");
const sandbox = new DenoSandboxExecutor({ timeout: 5000 });

console.log("Sandbox ready for context injection demos");

## 1. Basic Context Injection

Inject simple values and use them in code:

In [ ]:
const code = `
// 'name' and 'age' are injected directly into scope
return {
  greeting: \`Hello \${name}, you are \${age} years old\`,
  isAdult: age >= 18
};
`;

const result = await sandbox.execute(code, {
  name: "Alice",
  age: 30,
});

console.log(JSON.stringify(result, null, 2));

## 2. Processing API-like Data

A common use case: process an API response to extract relevant information:

In [ ]:
// Simulate an API response
const apiResponse = {
  users: [
    { id: 1, name: "Alice", email: "alice@example.com", active: true },
    { id: 2, name: "Bob", email: "bob@example.com", active: false },
    { id: 3, name: "Charlie", email: "charlie@example.com", active: true },
  ],
  metadata: {
    total: 3,
    timestamp: "2025-01-15T10:30:00Z",
  },
};

const code = `
// 'users' and 'metadata' are injected as variables
const activeUsers = users.filter(u => u.active);

return {
  activeCount: activeUsers.length,
  activeEmails: activeUsers.map(u => u.email),
  summary: \`\${activeUsers.length} out of \${metadata.total} users are active\`
};
`;

const result = await sandbox.execute(code, apiResponse);
console.log(JSON.stringify(result, null, 2));

## 3. Data Transformation Pipeline

Transform large datasets into summaries (reducing context usage):

In [ ]:
// Sales data that would be too large to put in LLM context
const salesData = {
  transactions: [
    { product: "Laptop", price: 1200, quantity: 2, date: "2025-01-10" },
    { product: "Mouse", price: 25, quantity: 5, date: "2025-01-11" },
    { product: "Keyboard", price: 80, quantity: 3, date: "2025-01-12" },
    { product: "Monitor", price: 300, quantity: 1, date: "2025-01-13" },
  ],
};

const code = `
// 'transactions' is injected - process it locally
const totals = transactions.map(t => ({
  ...t,
  total: t.price * t.quantity
}));

const grandTotal = totals.reduce((sum, t) => sum + t.total, 0);
const avgTransaction = grandTotal / totals.length;

const bestSeller = totals.reduce((best, current) => 
  current.total > best.total ? current : best
);

// Return only the summary (not the full dataset)
return {
  summary: {
    grandTotal,
    avgTransaction: Math.round(avgTransaction * 100) / 100,
    transactionCount: totals.length,
    bestSeller: { product: bestSeller.product, revenue: bestSeller.total }
  }
};
`;

const result = await sandbox.execute(code, salesData);
console.log("Input size:", JSON.stringify(salesData).length, "bytes");
console.log("Output size:", JSON.stringify(result.result).length, "bytes");
console.log();
console.log(JSON.stringify(result, null, 2));

## 4. Configuration-Driven Execution

Use context to control behavior:

In [ ]:
const config = {
  data: [10, 25, 30, 15, 40, 5],
  operation: "filter", // Try: 'filter', 'sort', 'transform'
  threshold: 20,
  sortOrder: "desc",
};

const code = `
let result = data;

switch (operation) {
  case 'filter':
    result = result.filter(x => x > threshold);
    break;
  case 'sort':
    result = [...result].sort((a, b) => 
      sortOrder === 'desc' ? b - a : a - b
    );
    break;
  case 'transform':
    result = result.map(x => x * 2);
    break;
}

return { operation, original: data, result, count: result.length };
`;

const result = await sandbox.execute(code, config);
console.log(JSON.stringify(result, null, 2));

## 5. Validating Context

Always validate injected data before using it:

In [ ]:
const code = `
// Validate before using
if (typeof data === 'undefined') {
  throw new Error("Missing required 'data' variable");
}

if (!Array.isArray(data)) {
  throw new Error("'data' must be an array");
}

return {
  sum: data.reduce((a, b) => a + b, 0),
  count: data.length
};
`;

// Test with invalid context
console.log("Test 1: Invalid context (string instead of array)");
const result1 = await sandbox.execute(code, { data: "not an array" });
console.log(JSON.stringify(result1, null, 2));

// Test with valid context
console.log("\nTest 2: Valid context");
const result2 = await sandbox.execute(code, { data: [1, 2, 3, 4, 5] });
console.log(JSON.stringify(result2, null, 2));

---

## Summary

Context injection enables:

- **Direct variable injection** - Each context key becomes a variable in scope
- **Dynamic data processing** - Transform any JSON data safely
- **Configuration-driven code** - Control behavior via injected variables
- **API integration** - Process external responses in isolation
- **Validation** - Check variables exist and have correct types

---

## Quick Check

Before moving on:

1. **How do you pass data to the sandbox?**
   - As the second argument to `sandbox.execute(code, context)`

2. **How are context values accessed in code?**
   - As direct variable names (not `context.key`, just `key`)

3. **Why validate context inside the sandbox?**
   - To handle missing or malformed data gracefully

---

**Next:** [06-mcp-gateway.ipynb](./06-mcp-gateway.ipynb) - Connect to MCP servers through the
gateway